# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e10 ##5e5 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ObstacleCurriculum" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 #0.99 # Reward discount rate.
lambd = 0.95 ##0.95 # Lambda parameter for GAE.
time_horizon = 2048 ##2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 ##1e-3 # Strength of entropy regularization
num_epoch = 5 ##5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 ##2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 ##0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 4096 ##2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 128 ##64 # Number of units in hidden layer.
batch_size = 32 ##64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'ObstacleCurriculumAcademy' started successfully!


Unity Academy name: ObstacleCurriculumAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: ObstacleCurriculumBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 9
        Action space type: discrete
        Action space size (per agent): 8
        Memory space size (per agent): 0
        Action descriptions: , , , , , , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/ppo\model-4750000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-4750000.cptk


Saved Model
Step: 4760000. Mean Reward: 0.6269222887423311. Std of Reward: 0.263868472253713.
Step: 4770000. Mean Reward: 0.6354545439272726. Std of Reward: 0.1747553715235771.
Step: 4780000. Mean Reward: 0.5607708316874997. Std of Reward: 0.5071980795649748.
Step: 4790000. Mean Reward: 0.52142404878481. Std of Reward: 0.5468130983625192.
Step: 4800000. Mean Reward: 0.5606646202638036. Std of Reward: 0.3540345194954728.
Saved Model
Step: 4810000. Mean Reward: 0.5890020561172838. Std of Reward: 0.4011562702179468.
Step: 4820000. Mean Reward: 0.6383028442499998. Std of Reward: 0.1753536373284906.
Step: 4830000. Mean Reward: 0.5193313358323353. Std of Reward: 0.5097785990492378.
Step: 4840000. Mean Reward: 0.505429446570552. Std of Reward: 0.5560592852113955.
Step: 4850000. Mean Reward: 0.5879245268427671. Std of Reward: 0.36753103383489494.
Saved Model
Step: 4860000. Mean Reward: 0.40098888698666657. Std of Reward: 0.7640101336915256.
Step: 4870000. Mean Reward: 0.4211037513576157. Std o

Step: 5720000. Mean Reward: 0.6008678477514792. Std of Reward: 0.3412060004752839.
Step: 5730000. Mean Reward: 0.5645759346923076. Std of Reward: 0.4249790198065954.
Step: 5740000. Mean Reward: 0.5846686724638553. Std of Reward: 0.36418511410286764.
Step: 5750000. Mean Reward: 0.45156378398765423. Std of Reward: 0.6395304261571326.
Saved Model
Step: 5760000. Mean Reward: 0.42598725899999984. Std of Reward: 0.7219770649693877.
Step: 5770000. Mean Reward: 0.5231612878774191. Std of Reward: 0.5493234079167104.
Step: 5780000. Mean Reward: 0.5223852276287424. Std of Reward: 0.5695817784974208.
Step: 5790000. Mean Reward: 0.4419461677142855. Std of Reward: 0.6699004957454036.
Step: 5800000. Mean Reward: 0.49540880311949675. Std of Reward: 0.6227245849561652.
Saved Model
Step: 5810000. Mean Reward: 0.5633835325240962. Std of Reward: 0.46437487389434134.
Step: 5820000. Mean Reward: 0.489404760220238. Std of Reward: 0.581691944661919.
Step: 5830000. Mean Reward: 0.4977510019939758. Std of Rewar

Step: 6680000. Mean Reward: 0.544926159620253. Std of Reward: 0.5229315067345478.
Step: 6690000. Mean Reward: 0.5577844305209578. Std of Reward: 0.4317884883826199.
Step: 6700000. Mean Reward: 0.48835305620118324. Std of Reward: 0.523277518992998.
Saved Model
Step: 6710000. Mean Reward: 0.49233539029012335. Std of Reward: 0.4966960266937366.
Step: 6720000. Mean Reward: 0.531142556798742. Std of Reward: 0.5196337046520345.
Step: 6730000. Mean Reward: 0.48161290254193534. Std of Reward: 0.6081662139851329.
Step: 6740000. Mean Reward: 0.5349363048662417. Std of Reward: 0.48087793771632653.
Step: 6750000. Mean Reward: 0.4830312492874998. Std of Reward: 0.5782501402044434.
Saved Model
Step: 6760000. Mean Reward: 0.45676406868181796. Std of Reward: 0.5912798028307474.
Step: 6770000. Mean Reward: 0.45249999926623363. Std of Reward: 0.6550748512257498.
Step: 6780000. Mean Reward: 0.4906458326312498. Std of Reward: 0.5442937890743381.
Step: 6790000. Mean Reward: 0.40632478559615365. Std of Rewa

Step: 7640000. Mean Reward: 0.562535352442424. Std of Reward: 0.4538420564712056.
Step: 7650000. Mean Reward: 0.5752268234911241. Std of Reward: 0.4102993943863346.
Saved Model
Step: 7660000. Mean Reward: 0.5246198822339179. Std of Reward: 0.49137558875499465.
Step: 7670000. Mean Reward: 0.5810404612658958. Std of Reward: 0.41877663419116107.
Step: 7680000. Mean Reward: 0.587321771219653. Std of Reward: 0.4023024069389819.
Step: 7690000. Mean Reward: 0.5550478914367815. Std of Reward: 0.4769474572245882.
Step: 7700000. Mean Reward: 0.4448412680535713. Std of Reward: 0.695402447994171.
Saved Model
Step: 7710000. Mean Reward: 0.4332638875297617. Std of Reward: 0.6569560225045852.
Step: 7720000. Mean Reward: 0.5218352045898875. Std of Reward: 0.5623993345253278.
Step: 7730000. Mean Reward: 0.5008234112916665. Std of Reward: 0.5498246472246191.
Step: 7740000. Mean Reward: 0.482725488717647. Std of Reward: 0.5706173610922507.
Step: 7750000. Mean Reward: 0.6059763297692305. Std of Reward: 0.

Step: 8600000. Mean Reward: 0.571104867696629. Std of Reward: 0.48409016310110625.
Saved Model
Step: 8610000. Mean Reward: 0.5839325827921347. Std of Reward: 0.40957795847866746.
Step: 8620000. Mean Reward: 0.5668659406576085. Std of Reward: 0.440221476258019.
Step: 8630000. Mean Reward: 0.5472644912880433. Std of Reward: 0.5063331449702483.
Step: 8640000. Mean Reward: 0.5549079177624309. Std of Reward: 0.4464741387529373.
Step: 8650000. Mean Reward: 0.5876684868306009. Std of Reward: 0.4040619477871966.
Saved Model
Step: 8660000. Mean Reward: 0.5985792337158469. Std of Reward: 0.3933757235550762.
Step: 8670000. Mean Reward: 0.5808110504919783. Std of Reward: 0.44186696629961614.
Step: 8680000. Mean Reward: 0.5975791419106143. Std of Reward: 0.3665842200128676.
Step: 8690000. Mean Reward: 0.5869809510742855. Std of Reward: 0.40081557283243585.
Step: 8700000. Mean Reward: 0.58673041752459. Std of Reward: 0.41895188867556893.
Saved Model
Step: 8710000. Mean Reward: 0.6150356491604277. St

error: unpack requires a bytes object of length 4

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-9350000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-9350000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
